In [1]:
import json
import math
import os
import random

import numpy as np
import pandas as pd

In [2]:
random.seed(4)

In [3]:
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks-in-python
def chunks(l, n):
    """Yield successive n-sized chunks from list l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [4]:
paths = os.listdir("test")
random.shuffle(paths) 

In [5]:
def extract_info(filename):
    nameonly = filename.split(".")[0]
    parts = nameonly.split("_")
    num = parts[0]
    mutation = False
    t = ""
    
    if parts[-1] == "halved" or parts[-1] == "zeroed":
        # Mutation present
        if parts[-1] == "zeroed":
            mutation = "zeroed"
            t = "_".join(parts[1:-1])
        else:
            mutation = "_".join(parts[-2:])
            t = "_".join(parts[1:-2])
    else:
        t = "_".join(parts[1:])
    return (num, t, mutation)

mutated = {}
mutated_list = []
regular = {}
regular_list = []
either = {}
either_list = []

for p in paths:
    (n, t, m) = extract_info(p)
    if t not in either:
        either[t] = []
    either[t].append(p)
    either_list.append(p)
    if m:
        if t not in mutated:
            mutated[t] = []
        mutated[t].append(p)
        mutated_list.append(p)
    else:
        if t not in regular:
            regular[t] = []
        regular[t].append(p)
        regular_list.append(p)
   

In [6]:
# Number of files a node will read from at a time
per = 5
num_nodes = 8

In [7]:
# assignment_iid
# 8 IID nodes
iid_assignment = list(chunks(either_list, 100))[:num_nodes]

for i in range(len(iid_assignment)):
    iid_assignment[i] = list(chunks(iid_assignment[i], per))

with open('assignment_iid.json', 'w') as outfile:
    json.dump(iid_assignment, outfile)

In [8]:
# assignment_non
# 8 nodes, very non-IID, each node has one kind of pattern only + mutations are bunched together
non_assignment = []

for k in regular.keys():
    regs = list(chunks(regular[k], per))
    muts = list(chunks(mutated[k], per))
    mixed = regs + muts
    random.shuffle(mixed)
    non_assignment.append(mixed)

non_assignment = non_assignment[:num_nodes]

with open('assignment_non.json', 'w') as outfile:
    json.dump(non_assignment, outfile)